<a href="https://colab.research.google.com/github/MandooMandooMan/FriendAI/blob/main/cagliostro-forge-colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![visitor][visitor-badge]][visitor-stats]

# **Cagliostro Forge Colab**
Rise from the ashes, reborn and empowered by [lllyasviel/stable-diffusion-webui-forge](https://github.com/lllyasviel/stable-diffusion-webui-forge)

**Version 1.0.0** | [Github][link-to-github] | [License](https://github.com/cagliostrolab/forge-colab/blob/main/LICENSE)

[visitor-badge]: https://api.visitorbadge.io/api/visitors?path=cagliostro-forge-colab&label=Visitors&labelColor=%2334495E&countColor=%231ABC9C&style=flat&labelStyle=none
[visitor-stats]: https://visitorbadge.io/status?path=cagliostro-forge-colab
[link-to-github]: https://github.com/cagliostrolab/forge-colab/blob/main/cagliostro-forge-colab.ipynb

In [1]:
# 🧼 Clean Pillow cache and reinstall correct version
!pip uninstall -y pillow
!pip install --no-cache-dir pillow==11.1.0
!pip install flask flask_cors pyngrok

Found existing installation: Pillow 9.5.0
Uninstalling Pillow-9.5.0:
  Successfully uninstalled Pillow-9.5.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 66.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
blendmodes 2022 requires Pillow<10,>=9.0.0, but you have pillow 11.1.0 which is incompatible.
gradio 4.40.0 requires pillow<11.0,>=8.0, but you have pillow 11.1.0 which is incompatible.


^C


In [1]:
!ngrok config add-authtoken 2uwYaZu77tzeXNUE43IfmblGdNW_6uKrRyLNzphkaoWQ6wqQf

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
# @title ## **Install Environment**
import sys
import subprocess
import os
import time
import json
import shutil
import random
import string
from pathlib import Path
from tqdm import tqdm
from pydantic import BaseModel

python_version  = ".".join(sys.version.split(".")[:2])
python_path     = Path(f"/usr/local/lib/python{python_version}/dist-packages/")
colablib_path   = python_path / "colablib"
if not colablib_path.exists():
    subprocess.run(['pip', 'install', '--upgrade', 'git+https://github.com/Linaqruf/colablib'], check=True)

from colablib.colored_print import cprint, print_line
from colablib.utils import py_utils, package_utils, config_utils
from colablib.sd_models.downloader import aria2_download, download
from colablib.utils.git_utils import update_repo, reset_repo, validate_repo, batch_update
from colablib.utils.py_utils import get_filename

################################
# COLAB ARGUMENTS GOES HERE
################################

# It ain't much, but it's honest work.
class CustomDirs(BaseModel):
    url: str
    dst: str

# @markdown ### **Drive Config**
mount_drive          = False  # @param {type: 'boolean'}
output_drive_folder  = "cagliostro-colab-forge"  # @param {type: 'string'}

# @markdown ### **Repo Config**
update_webui         = True  # @param {type: 'boolean'}
update_extensions    = True  # @param {type: 'boolean'}
commit_hash          = ""  # @param {type: 'string'}

# @markdown ### **Download Config**
# @markdown > Check only the options you need
animagine_xl_3_1     = False  # @param {type: 'boolean'}
rae_diffusion_xl_v2  = False  # @param {type: 'boolean'}
kivotos_xl_v2_0      = False  # @param {type: 'boolean'}
urangdiffusion_2_0   = False  # @param {type: 'boolean'}

# @markdown > **Note:**
# @markdown - For multiple URLs, use comma separation (e.g. `url1, url2, url3`)
# @markdown - Forge supports FLUX, SD, and SDXL, but this notebook focuses only on SDXL
# @markdown - **Highly Recommended:** Use Hugging Face links whenever possible
custom_model_url     = "https://huggingface.co/jwonlee95/test/blob/main/image%20model/anime/CalicoMix_PonyXL_v20.safetensors, https://huggingface.co/jwonlee95/test/blob/main/image%20model/realistic/ponyGeminixMix_v12.safetensors"  # @param {'type': 'string'}
custom_vae_url       = "https://huggingface.co/madebyollin/sdxl-vae-fp16-fix/resolve/main/sdxl.vae.safetensors"  # @param {'type': 'string'}
custom_lora_url      = ""  # @param {'type': 'string'}

# @markdown ### **Tunnel Config**
# @markdown > Default to `--share` until `ngrok_token` is not `None`
ngrok_token          = ""  # @param {type: 'string'}
ngrok_region         = "ap"  # @param ["us", "eu", "au", "ap", "sa", "jp", "in"]

# @markdown ### **UI/UX Config**
gradio_theme         = "remilia/Ghostly"  # @param ["Default", "gradio/base", "gradio/glass", "gradio/monochrome", "gradio/seafoam", "gradio/soft", "gradio/dracula_test", "abidlabs/dracula_test", "abidlabs/Lime", "abidlabs/pakistan", "Ama434/neutral-barlow", "dawood/microsoft_windows", "finlaymacklon/smooth_slate", "Franklisi/darkmode", "freddyaboulton/dracula_revamped", "freddyaboulton/test-blue", "gstaff/xkcd", "Insuz/Mocha", "Insuz/SimpleIndigo", "JohnSmith9982/small_and_pretty", "nota-ai/theme", "nuttea/Softblue", "ParityError/Anime", "reilnuud/polite", "remilia/Ghostly", "rottenlittlecreature/Moon_Goblin", "step-3-profit/Midnight-Deep", "Taithrah/Minimal", "ysharma/huggingface", "ysharma/steampunk", "NoCrypt/miku"]
# @markdown Set `use_preset` for using default prompt, resolution, sampler, and other settings
use_presets          = True  # @param {type: 'boolean'}

# @markdown ### **Launch Arguments**
use_gradio_auth      = False  # @param {type: 'boolean'}
auto_select_model    = False  # @param {type: 'boolean'}
auto_select_vae      = True  # @param {type: 'boolean'}
additional_arguments = "--api --lowram --theme dark --no-half-vae --opt-sdp-attention"  # @param {type: 'string'}

################################
# GLOBAL VARIABLES GOES HERE
################################

# GRADIO AUTH
user      = "cagliostro"
password  = "".join(random.choices(string.ascii_letters + string.digits, k=6))

# ROOT DIR
root_dir        = Path("/content")
drive_dir       = root_dir / "drive" / "MyDrive"
repo_dir        = root_dir / "stable-diffusion-webui-forge"
tmp_dir         = root_dir / "tmp"

models_dir      = repo_dir / "models"
extensions_dir  = repo_dir / "extensions"
ckpt_dir        = models_dir / "Stable-diffusion"
vae_dir         = models_dir / "VAE"
lora_dir        = models_dir / "Lora"
output_subdir   = ["txt2img-samples", "img2img-samples", "extras-samples", "txt2img-grids", "img2img-grids"]

config_file_path    = repo_dir / "config.json"
ui_config_file_path = repo_dir / "ui-config.json"

package_url = [
    "https://huggingface.co/Linaqruf/fast-repo/resolve/main/webui-forge.tar.lz4",
    "https://huggingface.co/Linaqruf/fast-repo/resolve/main/webui-forge-deps.tar.lz4",
]

custom_dirs = {
    "model" : CustomDirs(url=custom_model_url, dst=str(ckpt_dir)),
    "vae"   : CustomDirs(url=custom_vae_url, dst=str(vae_dir)),
    "lora"  : CustomDirs(url=custom_lora_url, dst=str(lora_dir)),
}

default_model_urls = {
    "animagine_xl_3_1"      : "https://huggingface.co/cagliostrolab/animagine-xl-3.1/resolve/main/animagine-xl-3.1.safetensors",
    "rae_diffusion_xl_v2"   : "https://huggingface.co/Raelina/Rae-Diffusion-XL-V2/resolve/main/RaeDiffusion-XL-v2.safetensors",
    "kivotos_xl_v2_0"       : "https://huggingface.co/yodayo-ai/kivotos-xl-2.0/resolve/main/kivotos-xl-2.0.safetensors",
    "urangdiffusion_2_0"    : "https://huggingface.co/kayfahaarukku/UrangDiffusion-2.0/resolve/main/UrangDiffusion-2.0.safetensors",
}

################################
# HELPER FUNCTIONS STARTS HERE
################################

def mount_drive_function(directory):
    output_dir = repo_dir / "outputs"

    if mount_drive:
        print_line(80, color="green")
        if not directory.exists():
            from google.colab import drive
            cprint("Mounting google drive...", color="green", reset=False)
            drive.mount(str(directory.parent))
        output_dir = directory / output_drive_folder
        cprint("Set default output path to:", output_dir, color="green")

    return output_dir

def setup_directories():
    for dir in [ckpt_dir, vae_dir, lora_dir]:
        dir.mkdir(parents=True, exist_ok=True)

def pre_download(dir, urls, desc, overwrite=False):
    ffmpy_path = python_path / "ffmpy-0.3.0.dist-info"

    for url in tqdm(urls, desc=desc):
        filename = Path(url).name
        aria2_download(dir, filename, url, quiet=True)
        if filename == "webui-forge-deps.tar.lz4":
            package_utils.extract_package(filename, python_path, overwrite=True)
        else:
            package_utils.extract_package(filename, "/", overwrite=overwrite)
        os.remove(dir / filename)

    subprocess.run(["rm", "-rf", str(ffmpy_path)])
    subprocess.run(["pip", "install", "--force-reinstall", "ffmpy"], check=True)

def install_dependencies():
    ubuntu_deps = ["aria2", "lz4"]
    cprint("Installing ubuntu dependencies", color="green")
    subprocess.run(["apt", "install", "-y"] + ubuntu_deps, check=True)

def install_webui(repo_dir, desc):
    if not repo_dir.exists():
        pre_download(root_dir, package_url, desc, overwrite=False)
    else:
        cprint("Stable Diffusion Web UI Forge already installed, skipping...", color="green")

def configure_output_path(config_path, output_dir, output_subdir):
    try:
        config = config_utils.read_config(str(config_path))
    except (FileNotFoundError, json.JSONDecodeError):
        config = {}

    config_updates = {
        f"outdir_{subdir.split('-')[0]}_{'_'.join(subdir.split('-')[1:])}": str(output_dir / subdir)
        for subdir in output_subdir
    }
    config.update(config_updates)

    config_path.parent.mkdir(parents=True, exist_ok=True)

    config_utils.write_config(str(config_path), config)

    for dir in output_subdir:
        (output_dir / dir).mkdir(parents=True, exist_ok=True)

def prepare_environment():
    cprint("Preparing environment...", color="green")
    os.environ['PYTORCH_CUDA_ALLOC_CONF']   = "garbage_collection_threshold:0.9,max_split_size_mb:512"
    os.environ["TF_CPP_MIN_LOG_LEVEL"]      = "3"
    os.environ["PYTHONWARNINGS"]            = "ignore"

def custom_download(custom_dirs):
    filtered_urls = filter_dict_items(default_model_urls)

    for key, value in custom_dirs.items():
        urls = value.url.split(",")
        dst = value.dst

        if key == "model":
            urls.extend(filtered_urls)

        if urls[0]:
            print_line(80, color="green")
            cprint(f" [-] Downloading Custom {key}...", color="flat_yellow")

        for url in urls:
            url = url.strip()
            if url != "":
                print_line(80, color="green")
                if "|" in url:
                    url, filename = map(str.strip, url.split("|"))
                    if not filename.endswith((".safetensors", ".ckpt", ".pt", "pth")):
                        filename = filename + Path(get_filename(url)).suffix
                else:
                    filename = get_filename(url)

                download(url=url, filename=filename, dst=dst, quiet=False)

def filter_dict_items(dict_items):
    result_list = []
    for key, url in dict_items.items():
        if globals().get(key):
            result_list.append(url)
    return result_list

def auto_select_file(target_dir, config_key, file_types):
    valid_files = [f for f in os.listdir(target_dir) if f.endswith(file_types)]
    if valid_files:
        file_path = random.choice(valid_files)

        if Path(target_dir).joinpath(file_path).exists():
            config = config_utils.read_config(str(config_file_path))
            config[config_key] = file_path
            config_utils.write_config(str(config_file_path), config)
        return file_path
    else:
        return None

def ui_config_presets():
    preset_prompt = "masterpiece, best quality, very aesthetic, absurdres"
    preset_negative_prompt = "nsfw, lowres, (bad), text, error, fewer, extra, missing, worst quality, jpeg artifacts, low quality, watermark, unfinished, displeasing, oldest, early, chromatic aberration, signature, extra digits, artistic error, username, scan, [abstract]"

    return {
        "txt2img/Prompt/value"              : preset_prompt,
        "txt2img/Negative prompt/value"     : preset_negative_prompt,
        "img2img/Prompt/value"              : preset_prompt,
        "img2img/Negative prompt/value"     : preset_negative_prompt,
        "customscript/sampler.py/txt2img/Sampling method/value" : "Euler a",
        "customscript/sampler.py/txt2img/Sampling steps/value"  : 28,
        "customscript/sampler.py/txt2img/Scheduler/value"       : "Automatic",
    }

def ui_config_settings(ui_config_file: str):
    config = config_utils.read_config(str(ui_config_file))
    preset_config = ui_config_presets()

    for key, value in preset_config.items():
        config[key] = value

    config_utils.write_config(str(ui_config_file), config)

def general_config_presets(config_file: str, lora_dir: str, use_presets: bool, ui_config_file: str):
    config = config_utils.read_config(str(config_file))

    config.update({
        "CLIP_stop_at_last_layers"      : 2,
        "show_progress_every_n_steps"   : 10,
        "show_progressbar"              : True,
        "samples_filename_pattern"      : "[model_name]_[seed]",
        "show_progress_type"            : "Approx NN",
        "live_preview_content"          : "Prompt",
        "forge_preset"                  : "xl",
        "xl_t2i_width"                  : 832,
        "xl_t2i_height"                 : 1216,
        "xl_t2i_cfg"                    : 7,
        "xl_t2i_hr_cfg"                 : 7,
        "xl_t2i_sampler"                : "Euler a",
        "xl_t2i_scheduler"              : "Automatic",
        "gradio_theme"                  : gradio_theme,
    })

    config_utils.write_config(str(config_file), config)

    if use_presets:
        ui_config_settings(ui_config_file)

def is_valid(target_dir, file_types):
    return any(f.endswith(file_types) for f in os.listdir(target_dir))

def parse_args(config):
    args = []
    for k, v in config.items():
        if k.startswith("_"):
            args.append(f'"{v}"')
        elif isinstance(v, str):
            args.append(f'--{k}="{v}"')
        elif isinstance(v, bool) and v:
            args.append(f"--{k}")
        elif isinstance(v, (float, int)) and not isinstance(v, bool):
            args.append(f"--{k}={v}")
    return " ".join(args)

def main():
    global output_dir, auto_select_model, auto_select_vae

    ################################
    # MAIN EXECUTION
    ################################

    os.chdir(root_dir)
    start_time = time.time()
    output_dir = mount_drive_function(drive_dir)

    gpu_info    = py_utils.get_gpu_info(get_gpu_name=True)
    python_info = py_utils.get_python_version()
    torch_info  = py_utils.get_torch_version()

    print_line(80, color="green")
    cprint(f" [-] Current GPU: {gpu_info}", color="flat_yellow")
    cprint(f" [-] Python {python_info}", color="flat_yellow")
    cprint(f" [-] Torch {torch_info}", color="flat_yellow")
    print_line(80, color="green")

    try:
        install_dependencies()

        print_line(80, color="green")
        install_webui(repo_dir, cprint("Unpacking Web UI Forge", color="green", tqdm_desc=True))
        prepare_environment()

        configure_output_path(config_file_path, output_dir, output_subdir)

        print_line(80, color="green")
        if update_webui and not commit_hash:
            update_repo(cwd=repo_dir, args="-X theirs --rebase --autostash")
        elif commit_hash:
            reset_repo(repo_dir, commit_hash)

        setup_directories()

        repo_name, current_commit_hash, current_branch = validate_repo(repo_dir)
        cprint(f"Using '{repo_name}' repository...", color="green")
        cprint(f"Branch: {current_branch}, Commit hash: {current_commit_hash}", color="green")

        if update_extensions:
            print_line(80, color="green")
            batch_update(fetch=True, directory=extensions_dir, desc=cprint("Updating extensions", color="green", tqdm_desc=True))

        elapsed_time = py_utils.calculate_elapsed_time(start_time)
        print_line(80, color="green")
        cprint(f"Finished installation. Took {elapsed_time}.", color="flat_yellow")
    except Exception as e:
        cprint(f"An error occurred: {str(e)}", color="red")
        print_line(80, color="red")
        cprint("Setup failed. Please check the error message above and try again.", color="red")
        print_line(80, color="red")
        return

    start_time = time.time()

    custom_download(custom_dirs)

    elapsed_time = py_utils.calculate_elapsed_time(start_time)
    print_line(80, color="green")
    cprint(f"Download finished. Took {elapsed_time}.", color="flat_yellow")
    print_line(80, color="green")
    cprint(f"Launching '{repo_name}'", color="flat_yellow")
    print_line(80, color="green")

    if not is_valid(ckpt_dir, ('.ckpt', '.safetensors')):
        cprint(f"No checkpoints were found in the directory '{ckpt_dir}'.", color="yellow")
        url = "https://huggingface.co/cagliostrolab/animagine-xl-3.1/blob/main/animagine-xl-3.1.safetensors"
        filename = get_filename(url)
        aria2_download(url=url, download_dir=ckpt_dir, filename=filename)
        print_line(80, color="green")
        auto_select_model = True

    if not is_valid(vae_dir, ('.vae.pt', '.vae.safetensors', '.pt', '.ckpt')):
        cprint(f"No VAEs were found in the directory '{vae_dir}'.", color="yellow")
        url = "https://huggingface.co/madebyollin/sdxl-vae-fp16-fix/blob/main/sdxl.vae.safetensors"
        filename = get_filename(url)
        aria2_download(url=url, download_dir=vae_dir, filename=filename)
        print_line(80, color="green")
        auto_select_vae = True

    if auto_select_model:
        selected_model  = auto_select_file(ckpt_dir, "sd_model_checkpoint", ('.ckpt', '.safetensors'))
        cprint(f"Selected Model: {selected_model}", color="green")

    if auto_select_vae:
        selected_vae    = auto_select_file(vae_dir, "sd_vae", ('.vae.pt', '.vae.safetensors', '.pt', '.ckpt'))
        cprint(f"Selected VAE: {selected_vae}", color="green")

    print_line(80, color="green")

    general_config_presets(config_file_path, lora_dir, use_presets, ui_config_file_path)

    if use_gradio_auth:
      cprint("Gradio Auth (use this account to login):", color="green")
      cprint("[-] Username: cagliostro", color="green")
      cprint("[-] Password:", password, color="green")
      print_line(80, color="green")

    config = {
        "enable-insecure-extension-access": True,
        "disable-safe-unpickle"           : True,
        "share"                           : True if not ngrok_token else False,
        "ngrok"                           : ngrok_token if ngrok_token else None,
        "ngrok-region"                    : ngrok_region if ngrok_token else None,
        "gradio-auth"                     : f"{user}:{password}" if use_gradio_auth else None,
        "no-hashing"                      : True,
        "disable-console-progressbars"    : True,
        "lowram"                          : True,
        "opt-sub-quad-attention"          : True,
        "opt-channelslast"                : True,
        "no-download-sd-model"            : True,
        "gradio-queue"                    : True,
        "listen"                          : True,
        "ckpt-dir"                        : ckpt_dir,
        "vae-dir"                         : vae_dir,
        "lora-dir"                        : lora_dir,
    }

    args = parse_args(config)
    final_args = f"python launch.py {args} {additional_arguments}"

    cprint()

    print("✅ WebUI setup done. Launching Flask + WebUI...")

    # Let launch happen after this function

    return final_args

from flask import Flask, request, jsonify, send_from_directory
from flask_cors import CORS
from threading import Thread
from pyngrok import ngrok
import requests
import base64
import random
import time

app = Flask(__name__)
CORS(app)

output_path = "/content/stable-diffusion-webui-forge/outputs/txt2img-samples"
os.makedirs(output_path, exist_ok=True)

model_map = {
    "anime": "CalicoMix_PonyXL_v20.safetensors",
    "realistic": "ponyGeminixMix_v12.safetensors"
}

@app.route("/outputs/<filename>")
def serve_image(filename):
    return send_from_directory(output_path, filename)

def generate_image(prompt: str, negative_prompt: str, model: str):
    try:
        # Switch model
        r = requests.post("http://127.0.0.1:7860/sdapi/v1/options", json={
            "sd_model_checkpoint": model
        })
        print("🔁 Model switch response:", r.status_code, r.text)
        r.raise_for_status()

        # Generate image
        res = requests.post("http://127.0.0.1:7860/sdapi/v1/txt2img", json={
            "prompt": prompt,
            "negative_prompt": negative_prompt,
            "steps": 28,
            "width": 832,
            "height": 1216,
            "sampler_name": "Euler a",
            "cfg_scale": 7
        })
        print("🖼️ Generation response:", res.status_code, res.text[:500])
        res.raise_for_status()

        image_data = res.json()["images"][0]

        seed = random.randint(100000, 999999)
        filename = f"{model.split('.')[0]}_{seed}.png"
        filepath = os.path.join(output_path, filename)

        with open(filepath, "wb") as f:
            f.write(base64.b64decode(image_data))

        return filename
    except Exception as e:
        print(f"[ERROR] Generation failed: {e}")
        return None

import base64

@app.route("/generate", methods=["POST"])
def generate():
    data = request.get_json()
    prompt = data.get("prompt")
    negative_prompt = data.get("negative_prompt", "")
    style = data.get("style", "anime")

    if not prompt:
        return jsonify({"error": "Prompt is required"}), 400
    if style not in model_map:
        return jsonify({"error": "Invalid style. Choose 'anime' or 'realistic'."}), 400

    print(f"✨ Generating image: '{prompt}' | Style: {style}")

    model = model_map[style]
    filename = generate_image(prompt, negative_prompt, model)

    if filename is None:
        return jsonify({"error": "Image generation failed"}), 500

    filepath = os.path.join(output_path, filename)
    with open(filepath, "rb") as f:
        image_data = f.read()
        base64_image = base64.b64encode(image_data).decode("utf-8")

    return jsonify({ "image": f"data:image/png;base64,{base64_image}" })

def start_flask():
    app.run(host="0.0.0.0", port=8001)

if __name__ == "__main__":
  # 🚫 Reset the repo to a clean state to avoid pull conflicts
    !cd /content/stable-diffusion-webui-forge && git reset --hard
    !cd /content/stable-diffusion-webui-forge && git clean -fd
    !cd /content/stable-diffusion-webui-forge && git checkout .

    final_args = main()  # now returns the WebUI launch command

    # Start Flask + ngrok
    public_url = ngrok.connect(8001)
    print("🌍 Your Flask API is live at:", public_url)
    Thread(target=start_flask).start()

    # Now run the blocking WebUI launch
    os.chdir(repo_dir)
    ! {final_args}

HEAD is now at c055f2d4 update bitsandbytes version for cuda128 support (#2712)
Updated 0 paths from the index
 [-] Current GPU: Tesla T4
 [-] Python 3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]
 [-] Torch 2.6.0+cu124
Installing ubuntu dependencies
Stable Diffusion Web UI Forge already installed, skipping...
Preparing environment...

Using 'lllyasviel/stable-diffusion-webui-forge' repository...
Branch: main, Commit hash: c055f2d43b07cbfd87ac3da4899a6d7ee52ebab9


Updating extensions: 100%|██████████| 5/5 [00:00<00:00,  5.77it/s]



Finished installation. Took 7 sec.
 [-] Downloading Custom model...
Starting download of 'CalicoMix_PonyXL_v20.safetensors' with aria2c...
Download of 'CalicoMix_PonyXL_v20.safetensors' completed. Took 0 sec.
Starting download of 'ponyGeminixMix_v12.safetensors' with aria2c...
Download of 'ponyGeminixMix_v12.safetensors' completed. Took 0 sec.
 [-] Downloading Custom vae...
Starting download of 'sdxl.vae.safetensors' with aria2c...
Download of 'sdxl.vae.safetensors' completed. Took 0 sec.
Download finished. Took 1 sec.
Launching 'lllyasviel/stable-diffusion-webui-forge'
Selected VAE: sdxl.vae.safetensors

✅ WebUI setup done. Launching Flask + WebUI...
🌍 Your Flask API is live at: NgrokTunnel: "https://5839-34-169-106-13.ngrok-free.app" -> "http://localhost:8001"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8001
 * Running on http://172.28.0.12:8001
INFO:werkzeug:Press CTRL+C to quit


Python 3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]
Version: f2.0.1v1.10.1-previous-659-gc055f2d4
Commit hash: c055f2d43b07cbfd87ac3da4899a6d7ee52ebab9
Installing requirements
Legacy Preprocessor init warning: Unable to install insightface automatically. Please try run `pip install insightface` manually.
Launching Web UI with arguments: --enable-insecure-extension-access --disable-safe-unpickle --share --no-hashing --disable-console-progressbars --lowram --opt-sub-quad-attention --opt-channelslast --no-download-sd-model --gradio-queue --listen --api --lowram --theme dark --no-half-vae --opt-sdp-attention
Total VRAM 15095 MB, total RAM 12979 MB
pytorch version: 2.6.0+cu124
Set vram state to: NORMAL_VRAM
Device: cuda:0 Tesla T4 : native
VAE dtype preferences: [torch.float32] -> torch.float32
CUDA Using Stream: False
E0000 00:00:1743791649.565490   42877 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been regi

INFO:werkzeug:127.0.0.1 - - [04/Apr/2025 18:35:56] "POST /generate HTTP/1.1" 200 -


🖼️ Generation response: 200 {"images":["iVBORw0KGgoAAAANSUhEUgAAA0AAAATACAIAAABWFrBDAAAA2HRFWHRwYXJhbWV0ZXJzAEtvcmVhbiBpZG9sClN0ZXBzOiAyOCwgU2FtcGxlcjogRXVsZXIgYSwgU2NoZWR1bGUgdHlwZTogQXV0b21hdGljLCBDRkcgc2NhbGU6IDcuMCwgU2VlZDogMjk4NjAyMjM5MSwgU2l6ZTogODMyeDEyMTYsIE1vZGVsOiBDYWxpY29NaXhfUG9ueVhMX3YyMCwgQ2xpcCBza2lwOiAyLCBWZXJzaW9uOiBmMi4wLjF2MS4xMC4xLXByZXZpb3VzLTY1OS1nYzA1NWYyZDQj02urAAEAAElEQVR4nNz9+fMlx3EnCLpHZL7jex91FwonQUKkSN1qnd2jnj5WMztmO7O9NmtrtvPT/k27a2u2M23W270z1t3bUmv6kNRqNSWeoAgCBAkSBEEQdxXq+tb3fEdm+P4QfkVmvm8V
✨ Generating image: 'Korean idol' | Style: anime
🔁 Model switch response: 200 null
[Unload] Trying to free 3302.87 MB for cuda:0 with 0 models keep loaded ... Current free memory is 9610.28 MB ... Done.
[Memory Management] Target: JointTextEncoder, Free GPU: 9610.28 MB, Model Require: 1752.98 MB, Previously Loaded: 0.00 MB, Inference Require: 1024.00 MB, Remaining: 6833.30 MB, All loaded to GPU.
Moving model(s) has taken 0.59 seconds
[Unload] Trying to free 1024.00 M

INFO:werkzeug:127.0.0.1 - - [04/Apr/2025 18:36:56] "POST /generate HTTP/1.1" 200 -


🖼️ Generation response: 200 {"images":["iVBORw0KGgoAAAANSUhEUgAAA0AAAATACAIAAABWFrBDAAAA2HRFWHRwYXJhbWV0ZXJzAEtvcmVhbiBpZG9sClN0ZXBzOiAyOCwgU2FtcGxlcjogRXVsZXIgYSwgU2NoZWR1bGUgdHlwZTogQXV0b21hdGljLCBDRkcgc2NhbGU6IDcuMCwgU2VlZDogMTc2NTY2MTUwMiwgU2l6ZTogODMyeDEyMTYsIE1vZGVsOiBDYWxpY29NaXhfUG9ueVhMX3YyMCwgQ2xpcCBza2lwOiAyLCBWZXJzaW9uOiBmMi4wLjF2MS4xMC4xLXByZXZpb3VzLTY1OS1nYzA1NWYyZDR2/m/oAAEAAElEQVR4nOz9d7glyXUfCJ5zIjLz+nuff1WvvGkPoGEaAOFIgZ6iEUmJpLSiE0WJs6K+/XZX3zcrrT7Nzq7MznyjWZnVUCSHWkmkZkSJHGroDbxrtDfV3VVd3tfz5vqbGXHO/hER


In [ ]:
# @title ## **Download Generated Images**
# @markdown Download file manually from files tab or save to Google Drive
import zipfile
from pathlib import Path
from google.colab import auth
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials
from colablib.colored_print import cprint

os.chdir(output_dir)

use_drive = False  # @param {type:"boolean"}
folder_name = "cagliostro-forge-colab"  # @param {type: "string"}
filename = "waifu.zip"  # @param {type: "string"}
save_as = filename

def get_unique_filename(base_filename):
    path = Path(base_filename)
    if not path.exists():
        return path
    i = 1
    while True:
        new_path = path.with_name(f"{path.stem}({i}){path.suffix}")
        if not new_path.exists():
            return new_path
        i += 1

filename = get_unique_filename(filename)

def zip_directory(directory, zipname):
    with zipfile.ZipFile(zipname, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for file_path in directory.rglob('*'):
            if file_path.is_file():
                zipf.write(file_path, file_path.relative_to(directory.parent))

zip_directory(output_dir, Path('/content/outputs.zip'))

if use_drive:
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive_service = GoogleDrive(gauth)

    def create_folder(folder_name):
        query = f"title='{folder_name}' and mimeType='application/vnd.google-apps.folder' and trashed=false"
        file_list = drive_service.ListFile({"q": query}).GetList()
        if file_list:
            cprint("Debug: Folder exists", color="green")
            return file_list[0]["id"]
        else:
            cprint("Debug: Creating folder", color="green")
            folder = drive_service.CreateFile({
                "title": folder_name,
                "mimeType": "application/vnd.google-apps.folder"
            })
            folder.Upload()
            return folder["id"]

    def upload_file(file_path, folder_id, save_as):
        save_as = get_unique_filename(save_as)
        file = drive_service.CreateFile({"title": save_as.name, "parents": [{"id": folder_id}]})
        file.SetContentFile(str(file_path))
        file.Upload()
        file.InsertPermission({"type": "anyone", "value": "anyone", "role": "reader"})
        return file["id"]

    folder_id = create_folder(folder_name)
    file_id = upload_file(Path('/content/outputs.zip'), folder_id, Path(save_as))
    sharing_link = f"https://drive.google.com/file/d/{file_id}/view?usp=sharing"
    cprint(f"Your sharing link: {sharing_link}", color="green")
else:
    cprint("Files zipped locally. Download manually from the files tab.", color="yellow")
